# Ensemble Model for Age Prediction

The provided Python code is an example of an **ensemble model** for age prediction from facial images. The idea behind an ensemble model is to combine the predictions of several base models to improve the robustness and performance of the prediction.

## Base Models

In this case, the base models are:

1. **ResNetModel**: This is likely a model based on the ResNet (Residual Network) architecture, which is a popular choice for image classification tasks due to its ability to train deep networks.

2. **InceptionModel**: This is likely a model based on the Inception architecture (also known as GoogLeNet), another popular choice for image classification tasks, known for its efficient use of model parameters.

3. **CNNModel**: This is a generic Convolutional Neural Network model. CNNs are widely used in image classification tasks due to their ability to capture spatial information.

## Ensemble Model

The `EnsembleModel` class is defined as a subclass of the `nn.Module` class, which is the base class for all neural network modules in PyTorch. The ensemble model takes a list of base models as input and stores them in an `nn.ModuleList`. This is necessary because PyTorch only optimizes the parameters of models that are instances of `nn.Module`.

In the `forward` method, the ensemble model computes the output of each base model and averages them. This is known as **model averaging**, a simple yet effective ensemble method. The assumption here is that each base model is equally reliable. If this is not the case, you might want to assign different weights to the outputs of different models.

Finally, an instance of the ensemble model is created with instances of the base models, and used for prediction on an input image.

Please note that the actual implementations of `ResNetModel`, `InceptionModel`, and `CNNModel` are not provided in the code. You would need to replace these with actual model classes for this code to run.

```python
# Create instances of your models
resnet_model = ResNetModel()
inception_model = InceptionModel()
cnn_model = CNNModel()

# Create an ensemble of the models
ensemble_model = EnsembleModel([resnet_model, inception_model, cnn_model])

# Use the ensemble model for prediction
input_image = # Load your input image
output = ensemble_model(input_image)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
from PIL import Image
from os.path import join
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import models, transforms

In [2]:

# Load datasets
class AgeDataset(Dataset):
    def __init__(self, data_path, annot_path, train=True):
        super(AgeDataset, self).__init__()
        self.annot_path = annot_path
        self.data_path = data_path
        self.train = train
        self.ann = pd.read_csv(annot_path)
        self.files = self.ann['file_id']
        if train:
            self.ages = self.ann['age']
        self.transform = transforms.Compose([
            transforms.Resize((299, 299)),  # Resize input images to 299x299
            transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
            transforms.RandomRotation(10),  # Randomly rotate the image by up to 10 degrees
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __getitem__(self, index):
        file_name = self.files[index]
        img = Image.open(join(self.data_path, file_name))
        img = self.transform(img)
        if self.train:
            age = self.ages[index]
            return img, age
        else:
            return img

    def __len__(self):
        return len(self.files)

train_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train'
train_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv'
train_dataset = AgeDataset(train_path, train_ann, train=True)

test_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test'
test_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv'
test_dataset = AgeDataset(test_path, test_ann, train=False)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)




In [3]:
def train_model_with_scheduler(model, train_loader, optimizer, criterion, scheduler, device, num_epochs=15):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            aux_logits, outputs = model(images)
            loss1 = criterion(outputs, labels.float().view(-1, 1))
            loss2 = criterion(aux_logits, labels.float().view(-1, 1))
            loss = loss1 + 0.4 * loss2  # Scale the auxiliary loss by 0.4 as done in the paper
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        scheduler.step(epoch_loss)  # Adjust learning rate based on epoch loss


def predict(loader, model, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images in tqdm(loader):
            images = images.to(device)
            outputs = model(images)
            predictions.extend(outputs.flatten().cpu().tolist())
    return predictions



In [ ]:
# Define the ResNet-based model architecture
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        # Load pre-trained ResNet model
        self.resnet = models.resnet18(pretrained=True)
        # Replace the final fully connected layer to adapt to your regression task
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 1)

    def forward(self, x):
        x = self.resnet(x)
        return x


In [ ]:
# Define the CNN model architecture
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 128 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class InceptionModel(nn.Module):
    def __init__(self):
        super(InceptionModel, self).__init__()
        self.inception = models.inception_v3(pretrained=True)
        # Handle the auxilary net
        num_ftrs = self.inception.AuxLogits.fc.in_features
        self.inception.AuxLogits.fc = nn.Linear(num_ftrs, 1)
        # Handle the primary net
        num_ftrs = self.inception.fc.in_features
        self.inception.fc = nn.Linear(num_ftrs, 1)

    def forward(self, x):
        if self.training:
            # In training mode, we return both outputs
            aux_logits, fc = self.inception(x)
            return aux_logits, fc
        else:
            # In evaluation mode, we only return the final output
            x = self.inception(x)
            return x

In [ ]:
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)
        
    def forward(self, x):
        outputs = torch.zeros(x.size(0), 1)
        for model in self.models:
            outputs += model(x)
        return outputs / len(self.models)

In [ ]:





# Check if GPU is available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create instances of your models
resnet_model = ResNetModel()
inception_model = InceptionModel()
cnn_model = CNNModel()

# Create an ensemble of the models
ensemble_model = EnsembleModel([resnet_model, inception_model, cnn_model])

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(ensemble_model.parameters(), lr=0.0001)  # Adjust learning rate if needed

# Learning rate scheduler (remain unchanged)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, threshold=1e-3, verbose=True)

# Train the model with learning rate scheduling
train_model_with_scheduler(ensemble_model, train_loader, optimizer, criterion, scheduler, device,15)

In [5]:
# Make predictions
predictions = predict(test_loader, ensemble_model, device)

# Create submission file
submission_df = pd.read_csv(test_ann)
submission_df['age'] = predictions
submission_df.to_csv('submission.csv', index=False)

100%|██████████| 61/61 [00:18<00:00,  3.21it/s]
